# Optimizing School District Assignments for Fairness, Efficiency, and Compactness

This notebook performs experiments using the Envy-Free and MILP optimization algorithms over multiple districts with varying parameters. We analyze performance metrics such as family cost, racial imbalance, compactness, entropy, and envy.

## Table of Contents
1. [Imports and Setup](#1)
2. [Load and Prepare Data](#2)
3. [Run Optimizations](#3)
4. [Grid Search over Lambda Values](#4)
5. [Visualizations and Analysis](#5)
6. [Conclusion](#6)

## 1. Imports and Setup <a class="anchor" id="1"></a>

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

# Import custom modules
from envy_free import eefx_allocation_composite, compute_metrics
from MILP_optimizer import solve_optimization
from preprocess_data import haversine_distance  # For recomputing missing distances

# Ensure plots are displayed in the notebook
%matplotlib inline

# Set Seaborn style for better aesthetics
sns.set(style='whitegrid')

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
random_seed = 42
np.random.seed(random_seed)

## 2. Load and Prepare Data <a class="anchor" id="2"></a>

In [12]:
# Paths to the preprocessed data files
blocks_file = 'gdf_blocks.pkl'
schools_file = 'gdf_schools.pkl'
dist_file = 'dist.pkl'

# Load the blocks DataFrame
gdf_blocks = pd.read_pickle(blocks_file)

# Load the schools DataFrame
gdf_schools = pd.read_pickle(schools_file)

# Load the distance dictionary
with open(dist_file, 'rb') as f:
    dist = pickle.load(f)

In [13]:
# Ensure 'UniqueTract' and 'UniqueSchoolID' exist and are strings
if 'UniqueTract' not in gdf_blocks.columns:
    gdf_blocks['UniqueTract'] = gdf_blocks['district_code'].astype(str) + '_' + gdf_blocks['Tract'].astype(str)
if 'UniqueSchoolID' not in gdf_schools.columns:
    gdf_schools['UniqueSchoolID'] = gdf_schools['district_code'].astype(str) + '_' + gdf_schools['SchoolID'].astype(str)

gdf_blocks['UniqueTract'] = gdf_blocks['UniqueTract'].astype(str)
gdf_schools['UniqueSchoolID'] = gdf_schools['UniqueSchoolID'].astype(str)

# Update block_names and school_names
block_names = gdf_blocks['UniqueTract'].unique().tolist()
school_names = gdf_schools['UniqueSchoolID'].unique().tolist()

In [14]:
# Verify that all blocks in gdf_blocks are present in dist
blocks_in_dist = set(dist.keys())
blocks_in_gdf = set(block_names)
missing_blocks = blocks_in_gdf - blocks_in_dist

if missing_blocks:
    print(f"Blocks missing in dist: {len(missing_blocks)}")
    print("Recomputing distances for missing blocks...")

    # Recompute distances for missing blocks
    for block_id in tqdm(missing_blocks, desc='Recomputing Distances'):
        block = gdf_blocks[gdf_blocks['UniqueTract'] == block_id].iloc[0]
        lat1, lon1 = block['Latitude'], block['Longitude']
        dist[block_id] = {}
        for _, school in gdf_schools.iterrows():
            school_id = school['UniqueSchoolID']
            lat2, lon2 = school['Latitude'], school['Longitude']
            distance = haversine_distance(lat1, lon1, lat2, lon2)
            dist[block_id][school_id] = distance

    # Verify all blocks are now present
    blocks_in_dist = set(dist.keys())
    missing_blocks = blocks_in_gdf - blocks_in_dist
    if missing_blocks:
        print(f"Still missing blocks in dist after recomputing: {len(missing_blocks)}")
    else:
        print("All blocks are now present in dist.")

In [15]:
# Compute overall racial proportions (P_k)
total_race_population = gdf_blocks[['White', 'Black', 'Asian']].sum().sum()
if total_race_population == 0:
    total_race_population = 1e-6  # Avoid division by zero

P_k = {
    'White': gdf_blocks['White'].sum() / total_race_population,
    'Black': gdf_blocks['Black'].sum() / total_race_population,
    'Asian': gdf_blocks['Asian'].sum() / total_race_population
}

print("Overall Racial Proportions (P_k):")
for race, proportion in P_k.items():
    print(f"{race}: {proportion:.4f}")

Overall Racial Proportions (P_k):
White: 0.9531
Black: 0.0376
Asian: 0.0093


In [16]:
# Calculate maximum possible racial imbalance
max_possible_imbalance = sum([abs(1 - P_k[race]) for race in P_k])

# Compute maximum distance for scaling
max_distance = max([max(dist_row.values()) for dist_row in dist.values() if dist_row])
BigM = max_distance + 1  # For compactness constraints

## 3. Run Optimizations <a class="anchor" id="3"></a>

In [17]:
# Parameters for the algorithms
lambda_distance = 1
lambda_racial = 1
lambda_capacity = 1
capacity_slack = 1.05  # Allow schools to exceed capacity by 5%

lambda_cost = 1
lambda_imbalance = 1
lambda_compactness = 1
solver_time_limit = 60000  # 60 seconds

In [18]:
# Run Envy-Free Algorithm with Composite Valuations
def run_envy_free():
    print("\nRunning Envy-Free Algorithm with Composite Valuations...")
    total_population = gdf_blocks['Population'].sum()
    num_schools = len(school_names)
    capacity_per_school = capacity_slack * (total_population / num_schools)

    allocation = eefx_allocation_composite(
        df_blocks=gdf_blocks,
        block_names=block_names,
        school_names=school_names,
        dist=dist,
        P_k=P_k,
        max_distance=max_distance,
        max_possible_imbalance=max_possible_imbalance,
        lambda_distance=lambda_distance,
        lambda_racial=lambda_racial,
        lambda_capacity=lambda_capacity,
        capacity_per_school=capacity_per_school,
        capacity_slack=capacity_slack
    )

    # Prepare results DataFrame
    assignments = []
    for school, blocks in allocation.items():
        for block in blocks:
            assignments.append({'UniqueTract': block, 'UniqueSchoolID': school})

    df_assignments = pd.DataFrame(assignments)

    # Merge assignments with block data
    df_results = pd.merge(df_assignments, gdf_blocks, on='UniqueTract', how='left')

    # Compute metrics
    metrics = compute_metrics(
        df_results=df_results,
        df_blocks=gdf_blocks,
        dist=dist,
        P_k=P_k,
        school_names=school_names,
        max_distance=max_distance
    )
    print("\nEnvy-Free Algorithm Metrics:")
    for key, value in metrics.items():
        print(f"{key}: {value}")

    return df_results, metrics

In [19]:
# Run MILP Optimization
def run_milp():
    print("\nRunning MILP Optimization...")
    result = solve_optimization(
        df_blocks=gdf_blocks,
        df_schools=gdf_schools,
        dist=dist,
        P_k=P_k,
        BigM=BigM,
        capacity_slack=capacity_slack,
        lambda_cost=lambda_cost,
        lambda_imbalance=lambda_imbalance,
        lambda_compactness=lambda_compactness,
        solver_time_limit=solver_time_limit
    )
    if result is None:
        print("No solution found.")
        return None, None
    df_results = result['assignments']
    metrics = {
        'family_cost': result['family_cost'],
        'racial_imbalance': result['racial_imbalance'],
        'compactness_penalty': result['compactness_penalty']
    }
    print("\nMILP Optimization Metrics:")
    for key, value in metrics.items():
        print(f"{key}: {value}")
    return df_results, metrics

In [ ]:
# Run both optimizations
df_eefx_results, eefx_metrics = run_envy_free()
df_milp_results, milp_metrics = run_milp()


Running Envy-Free Algorithm with Composite Valuations...


Allocating Blocks:  15%|█▍        | 364/2460 [03:04<16:31,  2.11it/s]

## 4. Grid Search over Lambda Values <a class="anchor" id="4"></a>

In [ ]:
# Parameters for grid search
lambda_values = [0.1, 0.5, 1, 2, 5]
district_codes = gdf_blocks['district_code'].unique().tolist()
capacity_slack = 1.05
solver_time_limit = 60000  # 60 seconds
parallel_processes = max(cpu_count() - 1, 1)  # Number of processes for parallel execution

In [ ]:
def run_experiments_for_district(district_code):
    district_results = []

    # Filter data for the district
    district_blocks = gdf_blocks[gdf_blocks['district_code'] == district_code]
    district_schools = gdf_schools[gdf_schools['district_code'] == district_code]
    block_names = district_blocks['UniqueTract'].unique().tolist()
    school_names = district_schools['UniqueSchoolID'].unique().tolist()

    # Filter dist for the district
    district_dist = {block: dist[block] for block in block_names}

    # Compute P_k for the district
    total_race_population = district_blocks[['White', 'Black', 'Asian']].sum().sum()
    if total_race_population == 0:
        total_race_population = 1e-6
    P_k = {
        'White': district_blocks['White'].sum() / total_race_population,
        'Black': district_blocks['Black'].sum() / total_race_population,
        'Asian': district_blocks['Asian'].sum() / total_race_population
    }
    max_possible_imbalance = sum([abs(1 - P_k[race]) for race in P_k])
    max_distance = max([max(dist_row.values()) for dist_row in district_dist.values() if dist_row])
    BigM = max_distance + 1

    total_population = district_blocks['Population'].sum()
    num_schools = len(school_names)
    capacity_per_school = capacity_slack * (total_population / num_schools)

    for lambda_value in tqdm(lambda_values, desc=f'District {district_code}'):
        # Envy-Free Algorithm
        allocation = eefx_allocation_composite(
            df_blocks=district_blocks,
            block_names=block_names,
            school_names=school_names,
            dist=district_dist,
            P_k=P_k,
            max_distance=max_distance,
            max_possible_imbalance=max_possible_imbalance,
            lambda_distance=lambda_value,
            lambda_racial=lambda_value,
            lambda_capacity=lambda_value,
            capacity_per_school=capacity_per_school,
            capacity_slack=capacity_slack
        )
        # Prepare results DataFrame
        assignments = []
        for school, blocks in allocation.items():
            for block in blocks:
                assignments.append({'UniqueTract': block, 'UniqueSchoolID': school})
        df_assignments = pd.DataFrame(assignments)
        df_results = pd.merge(df_assignments, district_blocks, on='UniqueTract', how='left')
        # Compute metrics
        metrics = compute_metrics(
            df_results=df_results,
            df_blocks=district_blocks,
            dist=district_dist,
            P_k=P_k,
            school_names=school_names,
            max_distance=max_distance
        )
        metrics.update({
            'district_code': district_code,
            'lambda_value': lambda_value,
            'algorithm': 'Envy-Free'
        })
        district_results.append(metrics)

        # MILP Optimization
        result = solve_optimization(
            df_blocks=district_blocks,
            df_schools=district_schools,
            dist=district_dist,
            P_k=P_k,
            BigM=BigM,
            capacity_slack=capacity_slack,
            lambda_cost=lambda_value,
            lambda_imbalance=lambda_value,
            lambda_compactness=lambda_value,
            solver_time_limit=solver_time_limit
        )
        if result is not None:
            metrics = {
                'family_cost': result['family_cost'],
                'racial_imbalance': result['racial_imbalance'],
                'compactness_penalty': result['compactness_penalty'],
                'district_code': district_code,
                'lambda_value': lambda_value,
                'algorithm': 'MILP'
            }
            district_results.append(metrics)
        else:
            print(f"MILP did not find a solution for district {district_code} with lambda {lambda_value}")

    return district_results

In [ ]:
# Run grid search in parallel
print("\nRunning Grid Search over Districts and Lambda Values...")
with Pool(processes=parallel_processes) as pool:
    all_results = []
    for district_results in tqdm(pool.imap_unordered(run_experiments_for_district, district_codes), total=len(district_codes)):
        all_results.extend(district_results)

In [ ]:
# Convert results to DataFrame
df_results = pd.DataFrame(all_results)

# Save results to CSV
df_results.to_csv('grid_search_results.csv', index=False)
print("\nGrid search results saved to 'grid_search_results.csv'.")

## 5. Visualizations and Analysis <a class="anchor" id="5"></a>

In [ ]:
# Function to plot metrics over lambda values
def plot_metric_over_lambda(df_results, metric, algorithm_name):
    plt.figure(figsize=(10, 6))
    sns.lineplot(
        data=df_results[df_results['algorithm'] == algorithm_name],
        x='lambda_value',
        y=metric,
        hue='district_code',
        marker='o'
    )
    plt.title(f"{metric.replace('_', ' ').title()} vs Lambda Value ({algorithm_name})")
    plt.xlabel('Lambda Value')
    plt.ylabel(metric.replace('_', ' ').title())
    plt.legend(title='District Code', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot metrics for Envy-Free algorithm
print("\nPlotting metrics for Envy-Free algorithm...")
for metric in ['family_cost', 'racial_imbalance', 'compactness_penalty', 'average_entropy', 'envy_count']:
    plot_metric_over_lambda(df_results, metric, 'Envy-Free')

In [ ]:
# Plot metrics for MILP algorithm
print("\nPlotting metrics for MILP algorithm...")
for metric in ['family_cost', 'racial_imbalance', 'compactness_penalty']:
    plot_metric_over_lambda(df_results, metric, 'MILP')

## 6. Conclusion <a class="anchor" id="6"></a>

In [ ]:
# Save the initial optimization results if needed
df_eefx_results.to_csv('eefx_initial_results.csv', index=False)
df_milp_results.to_csv('milp_initial_results.csv', index=False)
print("\nInitial optimization results saved to 'eefx_initial_results.csv' and 'milp_initial_results.csv'.")

**Summary:**

- We successfully ran both Envy-Free and MILP optimizations using the preprocessed data.
- We performed a grid search over different lambda values and districts.
- Progress bars were added to track long-running computations.
- The code handles distances and IDs consistently and efficiently.
- The results include assignments and computed metrics, which were visualized for analysis.